In [1]:
from icecube import dataio, hdfwriter, icetray, MuonGun, dataclasses
from icecube.sim_services.label_events import MCLabeler, MuonLabels
from icecube.sim_services.label_events import ClassificationConverter
from icecube.icetray import I3Tray
from icecube.dataclasses import I3Particle
from icecube.icetray import I3Units
from icecube.icetray import *
from icecube.sim_services.label_events.enums import classification
import numpy as np
import warnings

import numpy as np
from scipy.stats import norm
from scipy.special import erf
from scipy.stats import truncnorm
import pandas as pd
import os
import glob
# os.path.append('/mnt/home/baburish/jax/TriplePandelReco_JAX')
from _lib.pulse_extraction_from_i3 import get_pulse_info
from _lib.muon_energy import add_muon_energy

In [2]:
muon_label_list = []
def muonframe(frame):
  global muon_label_list
  try:
    seed = frame['coincident_muons'].value
    # if seed > 0:
    if frame['coincident_muons'] == True:
      False
    else:
      True
      # print(seed)
      muon_label_list.append(seed)
  except:
    print("Error")

In [3]:
RECOMPUTE_MU_E=True
def framestats(frame):
  global muon_label_list
  global event_header
  global interaction_type
  global most_energetic_track
  global primary_neutrino
  global spline_mpe
  global muon_energy_at_interaction
  global muon_energy_at_det
  global muon_energy_leaving
  global meta_frames
  global pulse_frames
  global event_count
  global pulse_data 
  global meta_data



  # if RECOMPUTE_MU_E:
  #           # Compute true properties of muon.
  #           #print("recomputing muon energy.")
  #           add_muon_energy(frame)
  try:
      muon_energy_at_interaction = frame[meta_keys['mc_muon_energy_at_interaction']].value # I3Double
      muon_energy_at_det =  frame[meta_keys['mc_muon_energy_at_detector_entry']].value # I3Double
      muon_energy_leaving = frame[meta_keys['mc_muon_energy_at_detector_leave']].value # I3Double
  except:
      print("Missing a key. Skip!")

  try:
    seed = frame['coincident_muons']
    event_header = frame['I3EventHeader']
    interaction_type = frame['I3MCWeightDict']['InteractionType']
    most_energetic_track = frame[meta_keys['mc_most_energetic_muon']] # I3Particle
    primary_neutrino = frame[meta_keys['mc_primary_neutrino']] # I3Particle
    spline_mpe = frame[meta_keys['spline_mpe']]
    muon_label_list.append(seed)
    # print("Loaded")
  except:
    print("Error")

  is_CC_interaction = interaction_type < 1.5
  pass_muon_energy = np.isfinite(muon_energy_at_det) and muon_energy_at_det > min_muon_energy_at_detector and muon_energy_at_det < max_muon_energy_at_detector
  energy_ratio = muon_energy_at_interaction  / most_energetic_track.energy
  found_correct_muon = energy_ratio < 0.9 or energy_ratio > 0.9
  has_sensible_muon = np.logical_and(pass_muon_energy, energy_ratio)
  if meta_keys['bkg_mc_tree'] == 'I3MCTree':
     has_no_coinc = len(frame['I3MCTree'].get_primaries()) == 1
  else:
     has_no_coinc = len(frame[meta_keys['bkg_mc_tree']]) == 0
  has_sensible_muon = np.logical_and(has_sensible_muon, has_no_coinc)
  if np.logical_and(is_CC_interaction, has_sensible_muon):
    # Retain event.
    event_count += 1

    event_id = event_header.run_id * n_events_per_file + event_header.event_id

    # Get all pulses.
    event_pulse_data, summary = get_pulse_info(frame, event_id, pulses_key=meta_keys['pulses'])
    print(event_id, summary)
    # Store.
    for key in pulse_data.keys():
        pulse_data[key] += event_pulse_data[key]

    # Get meta_data.
    event_last_pulse_idx = event_first_pulse_idx + summary['n_pulses'] - 1
    meta_data['event_id'].append(event_id)
    meta_data['idx_start'].append(event_first_pulse_idx)
    meta_data['idx_end'].append(event_last_pulse_idx)

    meta_data['neutrino_energy'].append(primary_neutrino.energy)
    meta_data['muon_energy'].append(muon_energy_at_interaction)
    meta_data['muon_energy_at_detector'].append(muon_energy_at_det)

    if np.isfinite(muon_energy_leaving):
        meta_data['muon_energy_lost'].append(muon_energy_at_det - muon_energy_leaving)
    else:
        # lost all energy inside the detector
        meta_data['muon_energy_lost'].append(muon_energy_at_det)

    meta_data['q_tot'].append(summary['q_tot'])
    meta_data['n_channel'].append(summary['n_channel'])
    meta_data['n_channel_HLC'].append(summary['n_channel_HLC'])
    meta_data['muon_zenith'].append(most_energetic_track.dir.zenith)
    meta_data['muon_azimuth'].append(most_energetic_track.dir.azimuth)
    meta_data['muon_time'].append(most_energetic_track.time)
    meta_data['muon_pos_x'].append(most_energetic_track.pos.x)
    meta_data['muon_pos_y'].append(most_energetic_track.pos.y)
    meta_data['muon_pos_z'].append(most_energetic_track.pos.z)
    meta_data['spline_mpe_zenith'].append(spline_mpe.dir.zenith)
    meta_data['spline_mpe_azimuth'].append(spline_mpe.dir.azimuth)
    meta_data['spline_mpe_time'].append(spline_mpe.time)
    meta_data['spline_mpe_pos_x'].append(spline_mpe.pos.x)
    meta_data['spline_mpe_pos_y'].append(spline_mpe.pos.y)
    meta_data['spline_mpe_pos_z'].append(spline_mpe.pos.z)


In [4]:
def label_muons(file):
    i3file = file
    gcd = "/cvmfs/icecube.opensciencegrid.org/data/GCD/GeoCalibDetectorStatus_2020.Run134142.Pass2_V0.i3.gz"
    tray = I3Tray()
    tray.Add("I3Reader", Filenamelist=[gcd, i3file])
    tray.Add(
    MCLabeler,
    event_properties_name=None,
    mctree_name='I3MCTree_preMuonProp',
    weight_dict_name='I3MCWeightDict',
    bg_mctree_name="I3MCTree_preMuonProp",
    )
    tray.Add(muonframe, Streams=[icetray.I3Frame.Physics])
    tray.Add(framestats, Streams=[icetray.I3Frame.Physics])
    tray.Execute()
    tray.PrintUsage()
    return meta_data, pulse_data, event_count

In [5]:
dataset_id = "22645"
file_index_start = 0
file_index_end = 10
outdir = "/mnt/home/baburish/jax/TriplePandelReco_JAX/extract_data_from_i3files/test"
os.makedirs(outdir, exist_ok=True)

directory = "/mnt/research/IceCube/Gupta-Reco/22645/0000000-0000999"
pattern = "FinalLevel_NuMu_NuGenCCNC.022645.000000.i3.zst"
file_pattern = os.path.join(directory, pattern)

i3files = sorted(glob.glob(file_pattern))
pulse_frames = []
meta_frames = []
i3files

['/mnt/research/IceCube/Gupta-Reco/22645/0000000-0000999/FinalLevel_NuMu_NuGenCCNC.022645.000000.i3.zst']

In [6]:
total_event_count = 0
for i, i3file in enumerate(i3files):
    n_events_per_file = int(1.e5)
    event_count = 0
    event_first_pulse_idx = 0 # inclusive
    event_last_pulse_idx = 0 # inclusive
    meta_keys = dict()
    meta_keys['pulses'] = 'TWSRTHVInIcePulsesIC'
    meta_keys['mc_primary_neutrino'] = 'MCPrimary1'
    meta_keys['mc_most_energetic_muon'] = 'MCMostEnergeticTrack'
    meta_keys['spline_mpe'] = 'SplineMPEIC'
    meta_keys['mc_muon_energy_at_interaction'] = 'TrueMuonEnergyAtInteraction'
    meta_keys['mc_muon_energy_at_detector_entry']  = 'TrueMuoneEnergyAtDetectorEntry'
    meta_keys['mc_muon_energy_at_detector_leave'] = 'TrueMuoneEnergyAtDetectorLeave'
    meta_keys['bkg_mc_tree'] = 'I3MCTree'
    pulse_data = {'event_id': [], 'sensor_id': [], 'time': [], 'charge': [], 'is_HLC':[]}

    meta_data = {'event_id': [], 'idx_start': [], 'idx_end': [], 'n_channel_HLC': []}
    meta_data.update({'neutrino_energy': [], 'muon_energy': [], 'muon_energy_at_detector': []})
    meta_data.update({'muon_energy_lost': [], 'q_tot': [], 'n_channel': []})
    meta_data.update({'muon_zenith': [], 'muon_azimuth': [], 'muon_time': []})
    meta_data.update({'muon_pos_x': [], 'muon_pos_y': [], 'muon_pos_z': []})
    meta_data.update({'spline_mpe_zenith': [], 'spline_mpe_azimuth': [], 'spline_mpe_time': []})
    meta_data.update({'spline_mpe_pos_x': [], 'spline_mpe_pos_y': [], 'spline_mpe_pos_z': []})
    min_muon_energy_at_detector = 1000 # GeV
    max_muon_energy_at_detector = 10000 # GeV
    print(f"Processing file {i+1}/{len(i3files)}: {i3file}")
    meta_data2, pulse_data2, event_count = label_muons(i3file)
    df_pulses = pd.DataFrame.from_dict(pulse_data2)
    df_meta = pd.DataFrame.from_dict(meta_data2)
    # print(df_pulses.head())
    pulse_frames.append(df_pulses)
    meta_frames.append(df_meta)
    print("Length of pulses", len(df_pulses))
    print("Len of pulse frames", len(pulse_frames))
    print("Length of meta frames", len(meta_frames))
    print("Event counts=", event_count)
    total_event_count += event_count

Processing file 1/1: /mnt/research/IceCube/Gupta-Reco/22645/0000000-0000999/FinalLevel_NuMu_NuGenCCNC.022645.000000.i3.zst
226450000000112 {'n_pulses': 72, 'n_channel': 39, 'n_channel_HLC': 27, 'q_tot': 77.750000461936}
226450000000148 {'n_pulses': 27, 'n_channel': 16, 'n_channel_HLC': 13, 'q_tot': 27.20000022649765}
226450000002879 {'n_pulses': 268, 'n_channel': 68, 'n_channel_HLC': 56, 'q_tot': 304.3500006645918}
226450000006369 {'n_pulses': 213, 'n_channel': 85, 'n_channel_HLC': 64, 'q_tot': 326.8749994635582}
226450000007878 {'n_pulses': 85, 'n_channel': 36, 'n_channel_HLC': 28, 'q_tot': 81.77499997615814}
---------------------------------------------------------------------------------------------------
                           I3Reader_0000:      0 calls to physics      0.61s user      0.05s system
                          MCLabeler_0000:    569 calls to physics      0.56s user      0.02s system
                         framestats_0000:      0 calls to physics      0.14s user 

NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


In [7]:
df_pulses2 = pd.concat(pulse_frames).reset_index(drop=True)
df_meta2 = pd.concat(meta_frames).reset_index(drop=True)
df_pulses2 = df_pulses2.drop_duplicates(subset=['event_id', 'sensor_id', 'time'])
df_meta2 = df_meta2.drop_duplicates(subset=['event_id'])

In [16]:
df_pulses2

,event_id,sensor_id,time,charge,is_HLC
0,226450000000112,319,14446.0,0.775,0
1,226450000000112,628,12667.0,1.175,0
2,226450000000112,684,13842.0,0.975,0
3,226450000000112,740,13684.0,0.875,1
4,226450000000112,742,13202.0,0.975,1
...,...,...,...,...,...
660,226450000007878,3361,12768.0,0.625,1
661,226450000007878,3361,12790.0,1.075,1
662,226450000007878,3361,12809.0,0.525,1
663,226450000007878,3361,12861.0,0.575,1


In [20]:
df_pulses2.iloc[41]

event_id     2.264500e+14
sensor_id    2.187000e+03
time         1.183300e+04
charge       9.175000e+00
is_HLC       1.000000e+00
Name: 41, dtype: float64

In [24]:
charge = df_pulses2[df_pulses2["event_id"] == 226450000000112]['charge']
aa = df_pulses2[df_pulses2["event_id"] == 226450000000112]
xx = charge.argmax()
aaa = aa[aa['sensor_id']==2187]
aaa

,event_id,sensor_id,time,charge,is_HLC
41,226450000000112,2187,11833.0,9.175,1
42,226450000000112,2187,11838.0,4.025,1
43,226450000000112,2187,11882.0,1.125,1
44,226450000000112,2187,12138.0,0.925,1
45,226450000000112,2187,12228.0,0.225,1
46,226450000000112,2187,12287.0,0.925,1
47,226450000000112,2187,12393.0,0.675,1


In [15]:

# for i, i3file in enumerate(i3files):
for i in range(0, 3):
    i3file = i3files[i]
    print(f"Processing file {i+1}/{len(i3files)}: {i3file}")
    meta_data2, pulse_data2, event_count = label_muons(i3file)
    df_pulses = pd.DataFrame.from_dict(pulse_data2)
    df_meta = pd.DataFrame.from_dict(meta_data2)
    pulse_frames.append(df_pulses)
    meta_frames.append(df_meta)
    print("Len of pframes", len(pulse_frames))
    print("Length of meta frames", len(meta_frames))
    print("Event counts=", event_count)
print("No of pulse frames", len(pulse_frames))
df_pulses2 = pd.concat(pulse_frames).reset_index(drop=True)
df_meta2 = pd.concat(meta_frames).reset_index(drop=True)

ofile_pulses = os.path.join(outdir, f"pulses_ds_{dataset_id}_from_{file_index_start}_to_{file_index_end}_10_to_100TeV.ftr")
ofile_meta   = os.path.join(outdir, f"meta_ds_{dataset_id}_from_{file_index_start}_to_{file_index_end}_10_to_100TeV.ftr")

df_pulses2.to_feather(ofile_pulses, compression='zstd')
df_meta2.to_feather(ofile_meta, compression='zstd')
print(f"Stored {event_count} events in outfiles:\n  {ofile_pulses}\n  {ofile_meta}")

Processing file 1/1: /mnt/research/IceCube/Gupta-Reco/22645/0000000-0000999/FinalLevel_NuMu_NuGenCCNC.022645.000000.i3.zst
226450000000112 {'n_pulses': 72, 'n_channel': 39, 'n_channel_HLC': 27, 'q_tot': 77.750000461936}
226450000000148 {'n_pulses': 27, 'n_channel': 16, 'n_channel_HLC': 13, 'q_tot': 27.20000022649765}
226450000002879 {'n_pulses': 268, 'n_channel': 68, 'n_channel_HLC': 56, 'q_tot': 304.3500006645918}
226450000006369 {'n_pulses': 213, 'n_channel': 85, 'n_channel_HLC': 64, 'q_tot': 326.8749994635582}
226450000007878 {'n_pulses': 85, 'n_channel': 36, 'n_channel_HLC': 28, 'q_tot': 81.77499997615814}
---------------------------------------------------------------------------------------------------
                          MCLabeler_0000:    569 calls to physics      0.62s user      0.04s system
                           I3Reader_0000:      0 calls to physics      0.61s user      0.04s system
                         framestats_0000:      0 calls to physics      0.14s user 

NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


Len of pframes 3
Length of meta frames 3
Event counts= 10


IndexError: list index out of range

In [16]:
df_pulses2

,event_id,sensor_id,time,charge,is_HLC
0,226450000000112,319,14446.0,0.775,0
1,226450000000112,628,12667.0,1.175,0
2,226450000000112,684,13842.0,0.975,0
3,226450000000112,740,13684.0,0.875,1
4,226450000000112,742,13202.0,0.975,1
...,...,...,...,...,...
4142,226450000207647,4596,13522.0,1.525,1
4143,226450000207647,4598,13225.0,1.275,1
4144,226450000207647,4599,13305.0,0.775,1
4145,226450000207647,4600,13506.0,0.775,1


In [18]:
n_unique_events = df_pulses2['event_id'].nunique()
print("Number of unique event IDs:", n_unique_events)

Number of unique event IDs: 15


In [9]:
df_subset = df_pulses
duplicate_row = df_subset.duplicated(["event_id", "sensor_id", "time", "charge", "is_HLC"], keep='first')
duplicate_rows = df_subset[duplicate_row]
num_duplicates = duplicate_row.sum()

print('num duplicates', num_duplicates)

num duplicates 0


In [9]:
df_subset.value_counts().reset_index(name='count').query("count > 1")

,event_id,sensor_id,time,charge,is_HLC,count
0,226450000000112,319,14446.0,0.775,0,3
1,226450000006369,3880,13200.0,0.475,1,3
2,226450000006369,3882,13275.0,0.775,1,3
3,226450000006369,3883,11959.0,1.275,1,3
4,226450000006369,3883,12028.0,1.025,1,3
...,...,...,...,...,...,...
1175,226450000105727,4131,12229.0,0.825,1,2
1176,226450000105727,4132,10441.0,1.875,1,2
1177,226450000105727,4132,10449.0,3.225,1,2
1178,226450000105727,4132,10475.0,0.425,1,2


In [21]:
dataset_id = "22645"
file_index_start = 0
file_index_end = 10
outdir = "/mnt/home/baburish/jax/TriplePandelReco_JAX/extract_data_from_i3files/test"
os.makedirs(outdir, exist_ok=True)

directory = "/mnt/research/IceCube/Gupta-Reco/22645/0000000-0000999"
pattern = "FinalLevel_NuMu_NuGenCCNC.022645.*.i3.zst"
file_pattern = os.path.join(directory, pattern)

i3files = sorted(glob.glob(file_pattern))
pulse_frames = []
meta_frames = []
total_event_count = 0
# for i, i3file in enumerate(i3files):
for i in range(0, 3):
    df_pulses = []
    df_meta = []
    pulse_frames = []
    meta_frames = []
    i3file = i3files[i]
    print(f"Processing file {i+1}/{len(i3files)}: {i3file}")
    meta_data2, pulse_data2, event_count = label_muons(i3file)
    df_pulses = pd.DataFrame.from_dict(pulse_data2)
    df_meta = pd.DataFrame.from_dict(meta_data2)

    pulse_frames.append(df_pulses)
    meta_frames.append(df_meta)
    print("Len of pframes", len(pulse_frames))
    print("Length of meta frames", len(meta_frames))
    total_event_count += event_count
print("No of pulse frames", len(pulse_frames))
df_pulses = pd.concat(pulse_frames).reset_index(drop=True)
df_meta = pd.concat(meta_frames).reset_index(drop=True)

ofile_pulses = os.path.join(outdir, f"pulses_ds_{dataset_id}_from_{file_index_start}_to_{file_index_end}_10_to_100TeV.ftr")
ofile_meta   = os.path.join(outdir, f"meta_ds_{dataset_id}_from_{file_index_start}_to_{file_index_end}_10_to_100TeV.ftr")

df_pulses.to_feather(ofile_pulses, compression='zstd')
df_meta.to_feather(ofile_meta, compression='zstd')
print(f"Stored {total_event_count} events in outfiles:\n  {ofile_pulses}\n  {ofile_meta}")

Processing file 1/1000: /mnt/research/IceCube/Gupta-Reco/22645/0000000-0000999/FinalLevel_NuMu_NuGenCCNC.022645.000000.i3.zst
---------------------------------------------------------------------------------------------------
                           I3Reader_0000:      0 calls to physics      0.61s user      0.05s system
                          MCLabeler_0000:    569 calls to physics      0.56s user      0.02s system
                         framestats_0000:      0 calls to physics      0.14s user      0.01s system
---------------------------------------------------------------------------------------------------
Len of pframes 1
Length of meta frames 1
Processing file 2/1000: /mnt/research/IceCube/Gupta-Reco/22645/0000000-0000999/FinalLevel_NuMu_NuGenCCNC.022645.000001.i3.zst


NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


---------------------------------------------------------------------------------------------------
                           I3Reader_0000:      0 calls to physics      0.59s user      0.02s system
                          MCLabeler_0000:    520 calls to physics      0.53s user      0.01s system
                         framestats_0000:      0 calls to physics      0.13s user      0.00s system
---------------------------------------------------------------------------------------------------
Len of pframes 1
Length of meta frames 1
Processing file 3/1000: /mnt/research/IceCube/Gupta-Reco/22645/0000000-0000999/FinalLevel_NuMu_NuGenCCNC.022645.000002.i3.zst


NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


---------------------------------------------------------------------------------------------------
                           I3Reader_0000:      0 calls to physics      0.58s user      0.03s system
                          MCLabeler_0000:    524 calls to physics      0.53s user      0.01s system
                         framestats_0000:      0 calls to physics      0.13s user      0.01s system
---------------------------------------------------------------------------------------------------
Len of pframes 1
Length of meta frames 1
No of pulse frames 1
Stored 73 events in outfiles:
  /mnt/home/baburish/jax/TriplePandelReco_JAX/extract_data_from_i3files/test/pulses_ds_22645_from_0_to_10_10_to_100TeV.ftr
  /mnt/home/baburish/jax/TriplePandelReco_JAX/extract_data_from_i3files/test/meta_ds_22645_from_0_to_10_10_to_100TeV.ftr


NOTICE (I3Tray): I3Tray finishing... (I3Tray.cxx:526 in void I3Tray::Execute(bool, unsigned int))


In [22]:
df_subset = df_pulses
duplicate_row = df_subset.duplicated(["event_id", "sensor_id", "time", "charge", "is_HLC"], keep='first')
duplicate_rows = df_subset[duplicate_row]
num_duplicates = duplicate_row.sum()

print('num duplicates', num_duplicates)

num duplicates 2302
